<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: Nona Ghazizadeh<br>
   **Student ID**: 98171007<br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

In this notebook the dataset that we choose is datsets for startups, it has features like R&D spend, administration, marketing spend and state after that from these features we can get profit of the startups

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('./startups.csv')
df.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In the code below we separate the independent and dependent features, and take out dependent feature which is profit freature in our dataset and then we reshape it

In [2]:
y = np.array(df['Profit'].values.tolist())
y = y.reshape(len(y), 1)

In the code below we remove profit column, hence our dataset has only independent features.

In [3]:
df.drop(["Profit"], axis = 1, inplace = True)

In the code below we want to handke the feauter "State" with the one hor encoding. At first we replace California by 1, New York by 2, Florida by 3. 
</br>
After that we create 3 columns with the respective State names. Then, we replace the values of the particular State of the particular State column by one and others by zero. This means, in the column California, we will replace all the California values by 1 and the values New York, Florida by zero.
</br>
After this, we can drop our original column "State".
</br>
ps: There is no hierarchy relation between the labels

In [4]:
df.replace(to_replace=["California","New York", "Florida"], value=[1,2,3])

df["California"] = df.iloc[:, 3]
df["New York"] = df.iloc[:,3]
df["Florida"] = df.iloc[:,3]

df.loc[df["California"]!="California", "California"] = 0
df.loc[df["California"]=="California", "California"] = 1

df.loc[df["New York"]!="New York", "New York"] = 0
df.loc[df["New York"]=="New York", "New York"] = 1

df.loc[df["Florida"]!="Florida", "Florida"] = 0
df.loc[df["Florida"]=="Florida", "Florida"] = 1

df.drop(["State"], axis = 1, inplace = True)
df.head()

,R&D Spend,Administration,Marketing Spend,California,New York,Florida
0,165349.20,136897.80,471784.10,0,1,0
1,162597.70,151377.59,443898.53,1,0,0
2,153441.51,101145.55,407934.54,0,0,1
3,144372.41,118671.85,383199.62,0,1,0
4,142107.34,91391.77,366168.42,0,0,1


In the code below we define two methods that they do feature scaling, one of them is for independent variables that means for our X any the other one is for dependent variables means for our Y.

In [7]:
def feature_scaling_for_independent_variable(variables):
    for i in range(variables.shape[1]-3):
        variables[:,i] = (variables[:,i] - int(np.mean(variables[:,i])))/np.std(variables[:,i])
    return variables
    
def feature_scaling_for_dependent_variable(variables):
    variables = (variables - int(np.mean(variables)))/np.std(variables)
    return variables

In the code below we use methods that we define previously to perfom feature scaling and also we add the feature X0 = 1 for our bias.

In [8]:
x = np.array(df.values.tolist())
x = feature_scaling_for_independent_variable(x)
x = np.concatenate((x,np.ones((50, 1))), axis = 1)

y = feature_scaling_for_dependent_variable(y)

In the next two code blocks at first we define a fucntion for spliting our dataset to train and test and in the next code block we split our data using thsi function.

In [9]:
def split_data(x, y, test_size=0.2, random_state=0):
    np.random.seed(random_state)                 
    indices = np.random.permutation(len(x))       
    data_test_size = int(x.shape[0] * test_size)  

    train_indices = indices[data_test_size:]
    test_indices = indices[:data_test_size]
    x_train = x[train_indices]
    y_train = y[train_indices]
    x_test = x[test_indices]
    y_test = y[test_indices]
    return x_train, y_train, x_test, y_test

In [10]:
x_train, y_train, x_test, y_test = split_data(x,y)

In the code below we define our common regression class.
</br>
Parameters for regression class are learning_rate that a samll value needed for gradient decent, iteration that is the number of training iteration and regularization which will be set to None for linear regression, l1 for lasso regression , l2 for ridge regression and l1_l2 for elastic net.
</br>
The cost function is based on the formula we have, if our regularization is equal to None it means that we don't have any regularization term due to our method that is linear regression but in the other methods we add our regularization term.
</br>
The hypothesis function is based on the formula which is X.w + bias and if we add 1 to our x we don't need any bias term,
</br>
In the train function, m is the number of training samples, n is the number of features and w is our weights that we set initial weights for it. In the for loop in this function we do these steps:
<ol>
    <li>
         Find the predicted value through the hypothesis.
    </li>
    <li>
         Find the Cost function value.
    </li>
    <li>
         Find the derivation of weights.
    </li>
    <li>
         Apply Gradient Decent.
    </li>
</ol> 
The predict function predicts y for given X.
</br>
The mse_loss function calculate loss based on the formula for mean squared error

In [13]:
class Regression:
    
    def __init__(self, learning_rate, iteration, regularization):
        self.m = None
        self.n = None
        self.w = None
        
        self.lr = learning_rate
        self.it = iteration
        self.regularization = regularization
        
    def cost_function(self, y, y_pred):
        if self.regularization == None:
            return (1 / (2 * self.m)) * np.sum(np.square(y_pred - y))
        else:
            return (1 / (2 * self.m)) * np.sum(np.square(y_pred - y)) + self.regularization(self.w)
    
    def hypothesis(self, weights, x):
        return np.dot(x, weights)
    
    def train(self, x, y):     
        self.m = x.shape[0]
        self.n = x.shape[1]
        self.w = np.zeros((self.n , 1))
        
        for it in range(1, self.it + 1):
            y_pred = self.hypothesis(self.w, x)
            cost = self.cost_function(y, y_pred)
            if self.regularization == None:
                dw = (1 / self.m) * np.dot(x.T, (y_pred - y))
            else:
                dw = (1 / self.m) * np.dot(x.T, (y_pred - y)) + self.regularization.derivation(self.w)

            self.w = self.w - self.lr * dw
   
    def predict(self, x):
        y_pred = self.hypothesis(self.w, x)
        return y_pred
    
    def mse_loss(self, y, y_pred):
        return np.sum(np.square(y - y_pred).ravel()) / y.shape[0]

In the code below we create the regularization class we want
</br>
l1_regularization for lasso regression.
</br>
l2_regularization for ridge regression.
</br>
l1_l2_regularization for elastic net regression.

In [14]:
class l1_regularization:

    def __init__(self, lamda):
        self.lamda = lamda

    def __call__(self, weights):
        return self.lamda * np.sum(np.abs(weights))
    
    def derivation(self, weights):
        return self.lamda * np.sign(weights)
    
class l2_regularization:

    def __init__(self, lamda):
        self.lamda = lamda

    def __call__(self, weights):
        return self.lamda * np.sum(np.square(weights))
    
    def derivation(self, weights):
        return self.lamda * 2 * (weights)
    
class l1_l2_regularization:
    
    def __init__(self, lamda = 0.1, l_ratio = 0.5):
        self.lamda = lamda 
        self.l_ratio = l_ratio

    def __call__(self, weights):
        l1_contribution = self.l_ratio * self.lamda * np.sum(np.abs(weights))
        l2_contribution = (1 - self.l_ratio) * self.lamda * 0.5 * np.sum(np.square(weights))
        return (l1_contribution + l2_contribution)

    def derivation(self, weights):
        l1_derivation = self.lamda * self.l_ratio * np.sign(weights)
        l2_derivation = self.lamda * (1 - self.l_ratio) * weights
        return (l1_derivation + l2_derivation)

In the code below we implement our lasso regression class. (we use created l1 regualarization class). At first we define the hyperparameters we are going to use in this model, lamda is our regularization factor, learning_rate is a samll value needed for gradient decent and iteration is the number of training iteration
</br> 
For train and predict function we use train and predict implemented in the common regression class

In [15]:
class LassoRegression(Regression):
    
    def __init__(self, lamda, learning_rate, iteration):
        self.regularization = l1_regularization(lamda)
        super(LassoRegression, self).__init__(learning_rate, iteration, self.regularization)

    def train(self, x, y):
        return super(LassoRegression, self).train(x, y)
    
    def predict(self, x):
        return super(LassoRegression, self).predict(x)

In the code below we implement our lasso regression class. (we use created l2 regualarization class). At first we define the hyperparameters we are going to use in this model, lamda is our regularization factor, learning_rate is a samll value needed for gradient decent and iteration is the number of training iteration
</br> 
For train and predict function we use train and predict implemented in the common regression class

In [16]:
class RidgeRegression(Regression):

    def __init__(self, lamda, learning_rate, iteration):
        self.regularization = l2_regularization(lamda)
        super(RidgeRegression, self).__init__(learning_rate, iteration, self.regularization)

    def train(self, x, y):
        return super(RidgeRegression, self).train(x, y)
    
    def predict(self, x):
        return super(RidgeRegression, self).predict(x)

In the code below we implement our lasso regression class. (we use created l1_l2 regualarization class). At first we define the hyperparameters we are going to use in this model, lamda is our regularization factor, l_ratio is the ratio between lasso and ridge regression, learning_rate is a samll value needed for gradient decent and iteration is the number of training iteration
</br> 
For train and predict function we use train and predict implemented in the common regression class

In [17]:
class ElasticNetRegression(Regression):

    def __init__(self, lamda, l_ratio, learning_rate, iteration):
        self.regularization = l1_l2_regularization(lamda,l_ratio)
        super(ElasticNetRegression, self).__init__(learning_rate, iteration, self.regularization)
    
    def train(self, x, y):
        return super(ElasticNetRegression, self).train(x, y)
    
    def predict(self, x):
        return super(ElasticNetRegression, self).predict(x)

In the next four code blocks we define our parameters, train our model, then we predict and after that we calculate mse loss. (first method is linear regression, second method is lasso regression, third method is ridge regression and fourth method is elastic net regression)

In [18]:
linear_reg_param = {
    "learning_rate" : 0.1,
    "iteration" : 1000,
    "regularization": None   
}
linear_reg = Regression(**linear_reg_param)
linear_reg.train(x_train, y_train) 

linear_reg_y_pred_test = linear_reg.predict(x_test)
linear_reg_test_score = linear_reg.mse_loss(y_test, linear_reg_y_pred_test)
print("The linear regression MSE loss for the test data is: ", linear_reg_test_score)

The linear regression MSE loss for the test data is:  0.052448372024488696


In [19]:
lasso_param = {
    "lamda" : 0.1,
    "learning_rate" : 0.1,
    "iteration" : 1000
}
lasso_reg = LassoRegression(**lasso_param)
lasso_reg.train(x_train, y_train) 
lasso_y_pred = lasso_reg.predict(x_train)

lasso_y_pred_test = lasso_reg.predict(x_test)
lasso_test_score = lasso_reg.mse_loss(y_test, lasso_y_pred_test)
print("The lasso regresion MSE loss for the test data is: ", lasso_test_score)

The lasso regresion MSE loss for the test data is:  0.05100980844230587


In [20]:
ridge_param = {
    "lamda" : 0.1,
    "learning_rate" : 0.1,
    "iteration" : 1000
}
ridge_reg = RidgeRegression(**ridge_param)
ridge_reg.train(x_train,y_train) 
ridge_y_pred = ridge_reg.predict(x_train)

ridge_y_pred_test = ridge_reg.predict(x_test)
ridge_test_score = ridge_reg.mse_loss(y_test, ridge_y_pred_test)
print("The ridge regresion MSE loss for the test data is: ", ridge_test_score)

The ridge regresion MSE loss for the test data is:  0.1150218086768257


In [21]:
elastic_net_param = {
    "l_ratio" : 0.5,
    "lamda" : 0.1,
    "learning_rate" : 0.1,
    "iteration" : 1000
}
elastic_net_reg = ElasticNetRegression(**elastic_net_param)
elastic_net_reg.train(x_train, y_train) 
elastic_net_y_pred = elastic_net_reg.predict(x_train)

elastic_net_y_pred_test = elastic_net_reg.predict(x_test)
elastic_net_test_score = elastic_net_reg.mse_loss(y_test, elastic_net_y_pred_test)
print("The elastic net MSE loss of the test data is: ", elastic_net_test_score)

The elastic net MSE loss of the test data is:  0.05139510943363168


As we can see for mse loss in the four methods, lasso regression is the best method, then elastic net regression is better that the others, after that linear regression and at the end the ridge regression is good by comparing their mse loss we can get the order below (from the best one to the worst one):
<ol>
    <li>
        Lasso Regression
    </li>
    <li>
        Elastic Net Regression
    </li>
</ol>